## Utitlity Functions
Useful functions to solve problems

In [1]:
import math
import time


def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print( '%r  %2.2f ms' % (method.__name__, (te - ts) * 1000))
        return result
    return timed


def generate_primes_under(n=100):
    """
    Returns list of all prime numbers <= n
    """
    for possiblePrime in range(2, n + 1): # Assume number is prime until shown it is not. 
        isPrime = True
        for num in range(2, int(possiblePrime ** 0.5) + 1):
            if possiblePrime % num == 0:
                isPrime = False
                break        
        if isPrime:
            yield possiblePrime

            
def is_prime(n):
    if n == 2 or n == 3: return True
    if n < 2 or n%2 == 0: return False
    if n < 9: return True
    if n%3 == 0: return False
    r = int(n**0.5)
    f = 5
    while f <= r:
        if n%f == 0: return False
        if n%(f+2) == 0: return False
        f +=6
    return True

## Problem 60 - Prime pair sets

The primes 3, 7, 109, and 673, are quite remarkable. By taking any two primes and concatenating them in any order the result will always be prime. For example, taking 7 and 109, both 7109 and 1097 are prime. The sum of these four primes, 792, represents the lowest sum for a set of four primes with this property.

Find the lowest sum for a set of five primes for which any two primes concatenate to produce another prime.

In [2]:
from itertools import permutations, combinations


def is_remarkable_pair(p1_p2):
    if is_prime(int(''.join(map(str, p1_p2)))):
        if is_prime(int(''.join(map(str, p1_p2[::-1])))):
            return True
    return False

            
primes = tuple(generate_primes_under(10000))

@timeit
def solution(initial_no_of_primes=70):
    result = []
    
    p1_p2_r_pairs = [p for p in combinations(primes[:initial_no_of_primes], 2) if is_remarkable_pair(p)]
    for r_pair in p1_p2_r_pairs:
        K = []
        for prime in primes:
            if is_remarkable_pair((r_pair[0], prime)) and is_remarkable_pair((r_pair[1], prime)):
                K.append(prime)

        k1_k2_pairs = [k for k in combinations(K, 2) if is_remarkable_pair(k)]
        for k_pair in k1_k2_pairs:
            for k in K:
                if sorted((*r_pair, *k_pair, k)) in result:
                    continue
                if is_remarkable_pair((k_pair[0], k)) and is_remarkable_pair((k_pair[1], k)):
                    print(*r_pair, *k_pair, k)
                    result.append(sorted((*r_pair, *k_pair, k)))
                    # Remove return statement if you want all solutions
                    return result
        
    return result


# [13, 5197, 5701, 6733, 8389]
remarkable_primes = solution(1000)

def test_solution(remarkable_primes):
    for p in permutations(remarkable_primes, 2):
        number = int(str(p[0]) + str(p[1]))
        if not is_prime(number):
            print(number)
        assert is_prime(number)
        
(test_solution(s) for s in remarkable_primes)
min_sum = min([sum(s) for s in remarkable_primes])
min_r_set = [s for s in remarkable_primes if sum(s) == min_sum][0]
    
print(min_sum)
print(min_r_set)


13 5197 5701 6733 8389
'solution'  18700.89 ms
26033
[13, 5197, 5701, 6733, 8389]
